In [1]:
!nvidia-smi
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

/bin/bash: nvidia-smi: command not found
--2022-02-13 09:23:18--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-02-13 09:23:18--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-02-13 09:23:19--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [applicatio

In [3]:
import pandas as pd
import numpy as np
from collections import Counter
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
train_df = pd.read_csv('mami_training.csv', sep='\t', lineterminator='\r')
test_df = pd.read_csv('Test.csv', sep='\t', lineterminator='\n')
test_labels = pd.read_csv('test_labels.txt',header=None, sep='\t', lineterminator='\n')
X = train_df['Text Transcription'].copy()
for i in range(len(X)):
  X[i] = X[i].lower()
X_test_model = test_df['Text Transcription'].to_numpy()
for i in range(len(X_test_model)):
  X_test_model[i] = X_test_model[i].lower()
Y = train_df['misogynous']
X_train_model = X[:].to_numpy()
Y_train_model = Y[:].to_numpy()
print(X_train_model.shape, Y_train_model.shape)
print(Y_train_model.sum())

(10000,) (10000,)
5000


In [4]:
embeddings_index = {}

f = open('glove.6B.200d.txt')
for line in f:
    values = line.split()
    word = value = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' %len(embeddings_index))

Found 400000 word vectors.


In [5]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence  import pad_sequences

tokenizer_obj = Tokenizer()
total_memes =  train_df['Text Transcription']
tokenizer_obj.fit_on_texts(total_memes)

max_length = 50 #max([len(s.split()) for s in total_memes])
vocab_size  = len(tokenizer_obj.word_index) + 1
X_train_tokens = tokenizer_obj.texts_to_sequences(X_train_model)
X_test_tokens = tokenizer_obj.texts_to_sequences(X_test_model)
X_train_pad = pad_sequences(X_train_tokens, maxlen=max_length)
X_test_pad = pad_sequences(X_test_tokens, maxlen=max_length)
print(vocab_size)

20693


In [6]:
EMBEDDING_DIM = 200
word_index = tokenizer_obj.word_index
embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
      embedding_matrix[i] = embedding_vector

def average_glove(df):
  real_X = np.zeros((len(df) , EMBEDDING_DIM))
  for i in range(len(df)):
    num = 0
    for word in df[i]:
      num += 1
      real_X[i] += embedding_matrix[int(word)]
    if num!=0:
      real_X[i] = real_X[i]/num
  return real_X

X_train_pad_2_vec = average_glove(X_train_pad)
X_test_pad_2_vec = average_glove(X_test_pad)
print(X_test_pad_2_vec.shape,X_train_pad_2_vec.shape )

(1000, 200) (10000, 200)


In [7]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
Y_test_model = test_labels[1].to_numpy()
Y_train_model = train_df['misogynous'].to_numpy()
clf = KNeighborsClassifier(n_jobs=5)
clf.fit(X_train_pad_2_vec[:8000], Y_train_model[:8000])
print("######## Misogynous Prediction ########")
print("\nTraining Dataset ")
y_pred = clf.predict(X_train_pad_2_vec[:8000])
print(confusion_matrix(Y_train_model[:8000], y_pred),"\n F1 score - ", f1_score(Y_train_model[:8000], y_pred, average='macro'))
print("\nValidation Dataset ")
y_pred = clf.predict(X_train_pad_2_vec[8000:])
print(confusion_matrix(Y_train_model[8000:], y_pred),"\n F1 score - ", f1_score(Y_train_model[8000:], y_pred, average='macro'))
y_pred = clf.predict(X_test_pad_2_vec)
print("\nSubmission Score ")
print(confusion_matrix(Y_test_model, y_pred),"\n F1 score - ", f1_score(Y_test_model, y_pred, average='macro'))
subtaskA = f1_score(Y_test_model, y_pred, average='macro')
subtaskB = f1_score(Y_test_model, y_pred, average='macro')*(Y_test_model.sum())

######## Misogynous Prediction ########

Training Dataset 
[[2961 1002]
 [ 751 3286]] 
 F1 score -  0.7805127603173205

Validation Dataset 
[[618 419]
 [295 668]] 
 F1 score -  0.6427767354596623

Submission Score 
[[214 286]
 [128 372]] 
 F1 score -  0.5754002929122617


In [8]:
Y_train_model = train_df['shaming'].to_numpy()
Y_test_model = test_labels[2].to_numpy()
clf = KNeighborsClassifier(n_jobs=5)
clf.fit(X_train_pad_2_vec[:8000], Y_train_model[:8000])
print("######## Shaming Prediction ########")
print("\nTraining Dataset ")
y_pred = clf.predict(X_train_pad_2_vec[:8000])
print(confusion_matrix(Y_train_model[:8000], y_pred),"\n F1 score - ", f1_score(Y_train_model[:8000], y_pred, average='macro'))
print("\nValidation Dataset ")
y_pred = clf.predict(X_train_pad_2_vec[8000:])
print(confusion_matrix(Y_train_model[8000:], y_pred),"\n F1 score - ", f1_score(Y_train_model[8000:], y_pred, average='macro'))
y_pred = clf.predict(X_test_pad_2_vec)
print("\n######## Shaming Prediction ########")
print(confusion_matrix(Y_test_model, y_pred),"\n F1 score - ", f1_score(Y_test_model, y_pred, average='macro'))
subtaskB += f1_score(Y_test_model, y_pred, average='macro')*(Y_test_model.sum())

######## Shaming Prediction ########

Training Dataset 
[[6866   78]
 [ 854  202]] 
 F1 score -  0.6194190996076587

Validation Dataset 
[[1746   36]
 [ 200   18]] 
 F1 score -  0.5345241100732138

######## Shaming Prediction ########
[[827  27]
 [131  15]] 
 F1 score -  0.5361890000469682


In [9]:
Y_train_model = train_df['stereotype'].to_numpy()
Y_test_model = test_labels[3].to_numpy()
clf = KNeighborsClassifier(n_jobs=5)
clf.fit(X_train_pad_2_vec[:8000], Y_train_model[:8000])
print("######## Stereotype Prediction ########")
print("\nTraining Dataset ")
y_pred = clf.predict(X_train_pad_2_vec[:8000])
print(confusion_matrix(Y_train_model[:8000], y_pred),"\n F1 score - ", f1_score(Y_train_model[:8000], y_pred, average='macro'))
print("\nValidation Dataset ")
y_pred = clf.predict(X_train_pad_2_vec[8000:])
print(confusion_matrix(Y_train_model[8000:], y_pred),"\n F1 score - ", f1_score(Y_train_model[8000:], y_pred, average='macro'))
y_pred = clf.predict(X_test_pad_2_vec)
print("\n######## Stereotype Prediction ########")
print(confusion_matrix(Y_test_model, y_pred),"\n F1 score - ", f1_score(Y_test_model, y_pred, average='macro'))
subtaskB += f1_score(Y_test_model, y_pred, average='macro')*(Y_test_model.sum())

######## Stereotype Prediction ########

Training Dataset 
[[5100  484]
 [1171 1245]] 
 F1 score -  0.7305601103808128

Validation Dataset 
[[1351  255]
 [ 258  136]] 
 F1 score -  0.5934661370367802

######## Stereotype Prediction ########
[[534 116]
 [230 120]] 
 F1 score -  0.5824302079159647


In [10]:
Y_train_model = train_df['objectification'].to_numpy()
Y_test_model = test_labels[4].to_numpy()
clf = KNeighborsClassifier(n_jobs=5)
clf.fit(X_train_pad_2_vec[:8000], Y_train_model[:8000])
print("######## Objectification Prediction ########")
print("\nTraining Dataset ")
y_pred = clf.predict(X_train_pad_2_vec[:8000])
print(confusion_matrix(Y_train_model[:8000], y_pred),"\n F1 score - ", f1_score(Y_train_model[:8000], y_pred, average='macro'))
print("\nValidation Dataset ")
y_pred = clf.predict(X_train_pad_2_vec[8000:])
print(confusion_matrix(Y_train_model[8000:], y_pred),"\n F1 score - ", f1_score(Y_train_model[8000:], y_pred, average='macro'))
y_pred = clf.predict(X_test_pad_2_vec)
print("\n######## Objectification Prediction ########")
print(confusion_matrix(Y_test_model, y_pred),"\n F1 score - ", f1_score(Y_test_model, y_pred, average='macro'))
subtaskB += f1_score(Y_test_model, y_pred, average='macro')*(Y_test_model.sum())

######## Objectification Prediction ########

Training Dataset 
[[5880  287]
 [1147  686]] 
 F1 score -  0.6901332394657456

Validation Dataset 
[[1484  147]
 [ 288   81]] 
 F1 score -  0.5717641986009979

######## Objectification Prediction ########
[[570  82]
 [265  83]] 
 F1 score -  0.54511549740375


In [11]:
Y_train_model = train_df['violence'].to_numpy()
Y_test_model = test_labels[5].to_numpy()
clf = KNeighborsClassifier(n_jobs=5)
clf.fit(X_train_pad_2_vec[:8000], Y_train_model[:8000])
print("######## Violence Prediction ########")
print("\nTraining Dataset ")
y_pred = clf.predict(X_train_pad_2_vec[:8000])
print(confusion_matrix(Y_train_model[:8000], y_pred),"\n F1 score - ", f1_score(Y_train_model[:8000], y_pred, average='macro'))
print("\nValidation Dataset ")
y_pred = clf.predict(X_train_pad_2_vec[8000:])
print(confusion_matrix(Y_train_model[8000:], y_pred),"\n F1 score - ", f1_score(Y_train_model[8000:], y_pred, average='macro'))
y_pred = clf.predict(X_test_pad_2_vec)
print("\n######## Violence Prediction ########")
print(confusion_matrix(Y_test_model, y_pred),"\n F1 score - ", f1_score(Y_test_model, y_pred, average='macro'))
subtaskB += f1_score(Y_test_model, y_pred, average='macro')*(Y_test_model.sum())

######## Violence Prediction ########

Training Dataset 
[[7100   69]
 [ 643  188]] 
 F1 score -  0.6489207270891189

Validation Dataset 
[[1829   49]
 [ 111   11]] 
 F1 score -  0.5394861817072398

######## Violence Prediction ########
[[830  17]
 [133  20]] 
 F1 score -  0.5638266938063391


In [12]:
print("Subtask A score -", 
      subtaskA, 
      "\n Subtask B score - ", 
      subtaskB/(test_labels[5].sum()+ test_labels[4].sum()+ test_labels[3].sum()+test_labels[2].sum()+test_labels[1].sum()))

Subtask A score - 0.5754002929122617 
 Subtask B score -  0.5649966536288916
